In [ ]:
%reload_ext autoreload
%autoreload 2
#import os
#os.environ["MKL_NUM_THREADS"] = "1"
#os.environ["NUMEXPR_NUM_THREADS"] = "1"
#os.environ["OMP_NUM_THREADS"] = "1"

from pathlib import Path
home = str(Path.home())
import sys
import numpy as np
import matplotlib.pyplot as plt
import bct
import sys
sys.path.append(home + '/workspace/networkqit')
import networkqit as nq
from networkqit.graphtheory.models.MEModels import CWTECM, UBCM, UWCM, UECM3
from networkqit.utils.visualization import plot_mle
import pandas as pd
import pickle

In [ ]:
df_thresh_none = pickle.load(open('/home/carlo/workspace/entropy_analysis/notebooks/output/2019/processed/groups628/data_forcellini_thresh_none.pkl','rb'))['forcellini']
G = df_thresh_none[(df_thresh_none['motion']=='H') & (df_thresh_none['passages']==6)]['A'].values[0]#[0:64,0:64]
G = bct.threshold_proportional(G,0.05)
t = G[np.nonzero(G)].min()
print(t)

In [ ]:
W = G
A = (G>t).astype(float)
k = A.sum(axis=0)
s = G.sum(axis=0)
print('threshold=',t)

M = CWTECM(N=len(G), threshold=t)
x0 = (np.concatenate([k,s])) / max(np.max(k),np.max(s))
# Optimize by L-BFGS-B
opt = nq.MLEOptimizer(G, x0=x0, model=M)
import time
start = time.time()
sol = opt.run(model=M, verbose=2, maxiter=200, gtol=1E-6, method='MLE')
opt = nq.MLEOptimizer(G, x0=sol['x'], model=M)
sol = opt.run(model=M, verbose=2, maxiter=200, gtol=1E-6, method='saddle_point')
print('Optimization elapsed time = ', time.time()-start, ' seconds ')
print('Loglikelihood = ', M.loglikelihood(G,sol['x']))

pij = M.expected_adjacency(sol['x'])
wij = M.expected_weighted_adjacency(sol['x'])
plot_mle(G,pij,wij)

In [ ]:
print(np.sqrt((sol['jac']**2).sum()))
plt.plot(sol['jac'])

In [ ]:
import gzip
import seaborn as sns
df = pd.DataFrame(pickle.load(gzip.open('/home/carlo/workspace/entropy_analysis/notebooks/output/2019/processed/subjwise633/mle_dfinfo_subj_1.pkl.gz','rb')))

In [ ]:
df['loglikelihood'] = -df['loglikelihood']
grid = sns.FacetGrid(df,col='motion',hue='passages',sharey=True,sharex=True, margin_titles=True)
grid.map(plt.plot, 'thresh_prop','loglikelihood', marker='o').add_legend()

In [ ]:
df.drop(columns=['W','sol'])

In [ ]:
grid = sns.FacetGrid(df[df.passages==85],col='passages',hue='motion', margin_titles=True)
grid.map(plt.plot, 'thresh_prop','loglikelihood', marker='o').add_legend()

In [ ]:
for i in range(df.shape[0]):
    #plt.figure(figsize=(8,8))
    W = df['W'].values[i]
    M = CWTECM(N=len(W),G=W,threshold=df['thresh_abs'].values[i])
    title = 'passages %d motion %s thresh_prop %s' % (df['passages'].values[i],df['motion'].values[i],df['thresh_prop'].values[i])
    plot_mle(W,M.expected_adjacency(df['sol'].values[i]['x']), M.expected_weighted_adjacency(df['sol'].values[i]['x']), title=title)